In [ ]:
import cv2
import numpy as np
import yaml
import PIL.Image
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

# Load the overlay
base = BaseOverlay("base.bit")

In [ ]:
## Format frame for HDMI output ##

def format_frame(hdmi_out, frame):
    
    # Read input frame size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    
    # Create new video frame
    outframe = hdmi_out.newframe()
    
    # Fill new video frame with zero
    zero_img = np.zeros((480, 640), dtype=np.uint8)
    outframe[0:480,0:640] = zero_img[0:480,0:640]

    # Transfer input frame to the video frame
    # For Gray imge
    outframe[0:img_h,0:img_w] = frame[0:img_h,0:img_w]
    
    return outframe

In [ ]:
## Initialize HDMI I/O ##

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure HDMI input to gray scale
hdmi_in.configure(PIXEL_GRAY)
hdmi_in.start()

# Configure Output mode (w, h, bit per pixel)
hdmi_out_mode = VideoMode(640,480,8)
hdmi_out.configure(hdmi_out_mode, PIXEL_GRAY)
hdmi_out.start()

In [ ]:
## Prepare for calibration ##

# Set number of chessboard corners
chess_x = 5
chess_y = 9

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points
objp = np.zeros((chess_x*chess_y,3), np.float32)
objp[:,:2] = np.mgrid[0:chess_y,0:chess_x].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [ ]:
## Take 30 frame of chessboard for calibration ##

count = 30

while (count>0):
    # Read frame from HDMI input
    gray = hdmi_in.readframe()
    
    gray = cv2.resize(gray, (640,360))

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (chess_y,chess_x),None)

    # If chessboard found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the cornersn and wait 2s before taking next frame
        gray = cv2.drawChessboardCorners(gray, (chess_y,chess_x), corners2,ret)
        cv2.waitKey(2000)
        count = count - 1
    
    # Output to HDMI
    outframe = format_frame(hdmi_out, gray)
    hdmi_out.writeframe(outframe)
    print(count)

In [ ]:
## Calculate Calibration Parameter ##

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [ ]:
## Save Calibration Parameter to .yaml file ##

calibration_parameter = [ float(mtx[0,0]), float(mtx[0,2]), float(mtx[1,1]), float(mtx[1,2]), float(dist[0][0]), float(dist[0][1]), float(dist[0][2]), float(dist[0][3]), float(dist[0][4]) ]
fname = "calibration_parameter.yaml"

with open(fname, "w") as file:
    yaml.dump(calibration_parameter, file)

In [ ]:
## Load Calibration Parameter from saved file ##

fname = "calibration_parameter.yaml"
with open(fname) as file:
    data = yaml.load(file,Loader=yaml.Loader)
    
mtx = np.array([ [ data[0] , 0, data[1] ] , [ 0, data[2], data[3] ] , [0, 0, 1] ])
dist = np.array([ [ data[4], data[5], data[6], data[7], data[8] ] ])

In [ ]:
## Show frame after calibration##

while True:
    
    gray = hdmi_in.readframe()
    resized = cv2.resize(gray, (640,360))

    img = resized

    # undistort
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    outframe = format_frame(hdmi_out, dst)
    hdmi_out.writeframe(outframe)


In [ ]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out